# Kusto Detective Agency Season 1 (Challenge 2)

In this challenge, we need to spot election fraud. There's 4 candidates but a highly suspicious victory....
This challenge introdcude the bin function (binary). Finding high activity in short periods is very relevant to cyber. Is someone brute forcing after hours? Trying to DDoS? Have 500 errors spiked? Who knows... we do!

<hr>

## Solution

### Investigating the Data
We've brought in the Votes table, select the first 1000 to take a look<br>
![KDA_S1_C2_top.png](../../../images/Tools/BlueTeam/KQL/KDA_S1_C2_top.png)<br>

Taking a look at the first 1000, should start with the low hanging fruit. Is the voter hash repeated?<br>
![KDA_S1_C2_Hash.png](../../../images/Tools/BlueTeam/KQL/KDA_S1_C2_Hash.png)<br>

9 collisions in 5 million votes, I'll put that down to expected (and not enough for voter fraud). It could also be that someone is generating hashes randomly to brute force<br>
How about that voter IP column?<br>
![KDA_S1_C2_viaIP.png](../../../images/Tools/BlueTeam/KQL/KDA_S1_C2_viaIP.png)<br>

Normally I would think we're on to something here, but the hints tell us that votes are done via proxy in local areas. So once again this is probably expected.

### Introducing Bias
Lets introduce some bias and assume fraud did happen. In an election it's probably not the most moral thing; but in cyber there is a whole mindset of assuming you've been breached.<br>
Lets see where the votes came from, and order them by time.<br>
![KDA_S1_C2_Poppy1000.png](../../../images/Tools/BlueTeam/KQL/KDA_S1_C2_Poppy1000.png)<br>

OK, thats suspicious. So many votes in the first second of the polls opening, at the same address (which also happens to be the first URL). 10 seconds later it swaps to the next ip in the list and the same action repeats. Clearly something is sus here....<br>

### Calculating the fraud
So, someone is loading as many votes as possible to a server every 10 seconds, then moving on to the next. Let's work out the votes where there were more than 10, in a 10 second window, in a single IP. We can then count these and treat them as fake votes.<br>
Now, there may be perfectly legitimate votes in this time that we are bypassing. In the grand scheme, these are rounding errors so I'm not too worried about this.<br>
Time to introduce the bin function. Which does exactly that.<br>

![KDA_S1_C2_fakeVotes.png](../../../images/Tools/BlueTeam/KQL/KDA_S1_C2_fakeVotes.png)<br>

2.5 million votes? that looks like fraud to me.

### Removing the fraud
We can now work out how many votes each nominee had. Assuming there was no fraud for the others, they are just their own count, and we reduce the Poppy votes by the fraud amount.<br>
We have to multiply by 100 then divide over the total to get the percentage (cant get a partial int)<br>
![KDA_S1_C2_Final.png](../../../images/Tools/BlueTeam/KQL/KDA_S1_C2_Final.png)<br>
<br>
I really should pass the variable through here instead of running it in two separate calculations, but I honestly had trouble here. 

<hr>


## Final Script